## 1. Installation des dépendances

Installation des bibliothèques nécessaires.

In [2]:
!pip install -q sentence-transformers torch numpy requests

## 2. Connexion à Google Drive

Montage de Google Drive pour accéder aux documents.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

# Définir le chemin vers vos documents dans Google Drive
# Exemple: DRIVE_DATA_DIR = '/content/drive/MyDrive/rag_documents'
DRIVE_DATA_DIR = '/content/drive/MyDrive/m1_csmi_sgdb/data'

print(f"Google Drive monté. Répertoire de données: {DRIVE_DATA_DIR}")

Mounted at /content/drive
Google Drive monté. Répertoire de données: /content/drive/MyDrive/m1_csmi_sgdb/data


## 3. Configuration des providers LLM

Définition des différents providers compatibles OpenAI.

In [4]:
from dataclasses import dataclass
from typing import Optional, Dict
import os

@dataclass
class ProviderConfig:
    name: str
    url: str
    model: str
    api_key_env: Optional[str] = None

    def api_key(self) -> str:
        if not self.api_key_env:
            return ""
        return os.getenv(self.api_key_env, "")

# Registre des providers connus
PROVIDERS: Dict[str, ProviderConfig] = {
    "MISTRAL_LARGE": ProviderConfig(
        name="MISTRAL_LARGE",
        url="https://api.mistral.ai/v1/chat/completions",
        model="open-mistral-nemo",
        api_key_env="MISTRAL_API_KEY",
    ),
    "MISTRAL_CODESTRAL": ProviderConfig(
        name="MISTRAL_CODESTRAL",
        url="https://codestral.mistral.ai/v1/chat/completions",
        model="codestral-latest",
        api_key_env="CODESTRAL_API_KEY",
    ),
    "LOCAL_QWEN_CODER": ProviderConfig(
        name="LOCAL_QWEN_CODER",
        url="http://127.0.0.1:8080/v1/chat/completions",
        model="qwen3-32b-instruct",
        api_key_env=None,
    ),
    "IRMA_LLMCODE": ProviderConfig(
        name="IRMA_LLMCODE",
        url="http://llmcode.math.unistra.fr:8090/v1/chat/completions",
        model="qwen2.5-coder-instruct",
        api_key_env=None,
    ),
    "PALGANIA_QWEN3": ProviderConfig(
        name="PALGANIA_QWEN3",
        url="https://palgania.ovh:8106/v1/chat/completions",
        model="Qwen3-30B",
        api_key_env="TEXTSYNTH_API_KEY",
    ),
}

#DEFAULT_PROVIDER = "MISTRAL_CODESTRAL"
DEFAULT_PROVIDER = "MISTRAL_LARGE"


def get_provider(name: Optional[str] = None,
                 override_model: Optional[str] = None,
                 override_url: Optional[str] = None,
                 api_key: Optional[str] = None) -> ProviderConfig:
    """Retourne la configuration d'un provider."""
    if name is None:
        name = DEFAULT_PROVIDER
    if name not in PROVIDERS:
        raise ValueError(f"Provider inconnu: {name}. Providers disponibles: {list(PROVIDERS.keys())}")

    base = PROVIDERS[name]
    model = override_model or base.model
    url = override_url or base.url
    key = api_key if api_key is not None else base.api_key()

    return ProviderConfig(name=name, url=url, model=model, api_key_env=base.api_key_env)

print(f"Providers disponibles: {list(PROVIDERS.keys())}")

Providers disponibles: ['MISTRAL_LARGE', 'MISTRAL_CODESTRAL', 'LOCAL_QWEN_CODER', 'IRMA_LLMCODE', 'PALGANIA_QWEN3']


## 4. Configuration des clés API

Définissez vos clés API ici (pour Colab, utilisez les secrets).

In [5]:
import os

# Pour Google Colab, vous pouvez utiliser les secrets ou définir directement:
from google.colab import userdata
os.environ['CODESTRAL_API_KEY'] = userdata.get('CODESTRAL_API_KEY')
os.environ['MISTRAL_API_KEY'] = userdata.get('MISTRAL_API_KEY')

# Ou définir directement (⚠️ ne commitez jamais vos clés!):
# os.environ['MISTRAL_API_KEY'] = 'votre_clé_ici'
#os.environ['CODESTRAL_API_KEY'] = 'votre clé'
# os.environ['TEXTSYNTH_API_KEY'] = 'votre_clé_ici'

print("Clés API configurées (si nécessaire)")

Clés API configurées (si nécessaire)


## 5. Classe SimpleRAG

Implémentation complète du système RAG.

In [28]:
import glob
import numpy as np
from typing import List, Tuple, Optional
from sentence_transformers import SentenceTransformer
import requests
import urllib3

# Désactiver les avertissements SSL
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


class SimpleRAG:
    """Système RAG minimaliste avec sélection modulaire de provider LLM."""

    def __init__(
        self,
        data_dir: str = "data",
        provider_name: Optional[str] = None,
        override_model: Optional[str] = None,
        override_url: Optional[str] = None,
        api_key: Optional[str] = None,
    ):
        """Initialise le système RAG."""
        self.data_dir = data_dir
        self.documents = []
        self.embeddings = []

        # Modèle d'embedding
        print("Chargement du modèle d'embedding...")
        embed_model_name = "paraphrase-multilingual-MiniLM-L12-v2"
        self.embedding_model = SentenceTransformer(embed_model_name)
        print(f"  → Modèle {embed_model_name} chargé")

        # Configuration du provider LLM
        self._provider_cfg = get_provider(
            name=provider_name,
            override_model=override_model,
            override_url=override_url,
            api_key=api_key,
        )
        self.api_url = self._provider_cfg.url
        self.model = self._provider_cfg.model
        self.api_key = self._provider_cfg.api_key_env and os.getenv(self._provider_cfg.api_key_env, "") or ""

        print(f"Provider: {self._provider_cfg.name} | URL: {self.api_url} | Modèle: {self.model}")
        if self._provider_cfg.api_key_env:
            if not self.api_key:
                print(f"⚠️ Clé API manquante. Définissez '{self._provider_cfg.api_key_env}'")
            else:
                print(f"✓ Clé API chargée")

    def load_documents(self) -> None:
        """Charge tous les fichiers markdown du répertoire."""
        print(f"\nChargement des documents depuis {self.data_dir}/...")

        md_files = glob.glob(os.path.join(self.data_dir, "*.md"))

        for filepath in md_files:
            with open(filepath, 'r', encoding='utf-8') as f:
                content = f.read()
                paragraphs = [p.strip() for p in content.split('\n\n') if p.strip()]

                for para in paragraphs:
                    self.documents.append({
                        'text': para,
                        'source': os.path.basename(filepath)
                    })

        print(f"  → {len(self.documents)} chunks chargés depuis {len(md_files)} fichiers")

    def create_embeddings(self) -> None:
        """Génère les embeddings pour tous les documents."""
        print("\nCréation des embeddings...")

        texts = [doc['text'] for doc in self.documents]
        self.embeddings = self.embedding_model.encode(
            texts,
            show_progress_bar=True,
            convert_to_numpy=True
        )

        print(f"  → {len(self.embeddings)} embeddings créés (dimension: {self.embeddings.shape[1]})")

    def search(self, query: str, top_k: int = 3) -> List[Tuple[dict, float]]:
        """Recherche les documents les plus similaires à la requête."""
        query_embedding = self.embedding_model.encode(query, convert_to_numpy=True)

        similarities = np.dot(self.embeddings, query_embedding) / (
            np.linalg.norm(self.embeddings, axis=1) * np.linalg.norm(query_embedding)
        )

        top_indices = np.argsort(similarities)[-top_k:][::-1]

        results = [
            (self.documents[idx], float(similarities[idx]))
            for idx in top_indices
        ]

        return results

    def test_embedding(self):
      sentence1 = "le chat dort"
      sentence2 = "le matou fait la sieste"
      embed1= self.embedding_model.encode(sentence1, convert_to_numpy=True)
      embed2= self.embedding_model.encode(sentence2, convert_to_numpy=True)
      print(f"Similarity ('{sentence1}', '{sentence2}'): {np.dot(embed1, embed2) / (np.linalg.norm(embed1) * np.linalg.norm(embed2)):.4f}")

      sentence3 = "le chat ne dort pas"
      embed3= self.embedding_model.encode(sentence3, convert_to_numpy=True)
      print(f"Similarity ('{sentence1}', '{sentence3}'): {np.dot(embed1, embed3) / (np.linalg.norm(embed1) * np.linalg.norm(embed3)):.4f}")

      sentence4 = "la voiture est en panne"
      embed4= self.embedding_model.encode(sentence4, convert_to_numpy=True)
      print(f"Similarity ('{sentence1}', '{sentence4}'): {np.dot(embed1, embed4) / (np.linalg.norm(embed1) * np.linalg.norm(embed4)):.4f}")

      sentence5 = "the cat is sleeping"
      embed5= self.embedding_model.encode(sentence5, convert_to_numpy=True)
      print(f"Similarity ('{sentence1}', '{sentence5}'): {np.dot(embed1, embed5) / (np.linalg.norm(embed1) * np.linalg.norm(embed5)):.4f}")

    def configure_provider(
        self,
        provider_name: str,
        override_model: Optional[str] = None,
        override_url: Optional[str] = None,
        api_key: Optional[str] = None,
    ) -> None:
        """Change dynamiquement la configuration du provider LLM."""
        cfg = get_provider(
            name=provider_name,
            override_model=override_model,
            override_url=override_url,
            api_key=api_key,
        )
        self._provider_cfg = cfg
        self.api_url = cfg.url
        self.model = override_model or cfg.model
        self.api_key = api_key if api_key is not None else (cfg.api_key_env and os.getenv(cfg.api_key_env, "") or "")
        print(f"Provider reconfiguré: {provider_name} | URL: {self.api_url} | Modèle: {self.model}")

    def generate_with_llm(self, query: str, context_docs: List[dict]) -> str:
        """Génère une réponse en utilisant l'API REST."""
        context = "\n\n".join([doc['text'] for doc in context_docs])

        prompt = f"""Basé sur le contexte suivant (et uniquement sur ce contexte), réponds à la question de manière concise et précise.

Contexte:
{context}

Question: {query}

Réponse:"""

        #print(prompt)


        payload = {
            "model": self.model,
            "messages": [
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
            "temperature": 0.3,
            "max_tokens": 2000,
            "top_p": 0.9,
        }

        try:
            headers = {"Content-Type": "application/json"}
            if self.api_key:
                headers["Authorization"] = f"Bearer {self.api_key}"

            verify_ssl = self.api_url.startswith("https://")
            response = requests.post(
                self.api_url,
                json=payload,
                headers=headers,
                verify=verify_ssl,
                timeout=60,
            )
            response.raise_for_status()

            result = response.json()

            if "choices" in result and len(result["choices"]) > 0:
                return result["choices"][0]["message"]["content"].strip()
            else:
                return "Erreur : format de réponse inattendu de l'API"

        except requests.exceptions.RequestException as e:
            return f"Erreur lors de l'appel à l'API : {str(e)}"

## 6. Initialisation du système RAG

Créer une instance et charger les documents depuis Google Drive.

In [29]:
# Choisir le provider (None = DEFAULT_PROVIDER)
# Options: "MISTRAL_LARGE", "MISTRAL_CODESTRAL", "IRMA_LLMCODE", "PALGANIA_QWEN3"
PROVIDER = "MISTRAL_CODESTRAL"  # Changez selon vos besoins

# Initialisation
rag = SimpleRAG(data_dir=DRIVE_DATA_DIR, provider_name=PROVIDER)

# Chargement et indexation
rag.load_documents()
rag.create_embeddings()

Chargement du modèle d'embedding...
  → Modèle paraphrase-multilingual-MiniLM-L12-v2 chargé
Provider: MISTRAL_CODESTRAL | URL: https://codestral.mistral.ai/v1/chat/completions | Modèle: codestral-latest
✓ Clé API chargée

Chargement des documents depuis /content/drive/MyDrive/m1_csmi_sgdb/data/...
  → 1784 chunks chargés depuis 1 fichiers

Création des embeddings...


Batches:   0%|          | 0/56 [00:00<?, ?it/s]

  → 1784 embeddings créés (dimension: 384)


In [30]:
rag.test_embedding()

Similarity ('le chat dort', 'le matou fait la sieste'): 0.1057
Similarity ('le chat dort', 'le chat ne dort pas'): 0.8798
Similarity ('le chat dort', 'la voiture est en panne'): 0.1292
Similarity ('le chat dort', 'the cat is sleeping'): 0.9615


## 7. Tester une recherche simple

Rechercher des documents pertinents pour une question.

In [31]:
query = "Quel est le titre de ce roman ?"

print(f"Question: {query}\n")
print("="*70)

results = rag.search(query, top_k=3)

for i, (doc, score) in enumerate(results, 1):
    print(f"\n[{i}] Score: {score:.4f} | Source: {doc['source']}")
    print(f"    {doc['text'][:200]}...")

Question: Quel est le titre de ce roman ?


[1] Score: 0.6751 | Source: chartreuse_de_parme_stendhal.md
    Titre du roman: La Chartreuse de Parme...

[2] Score: 0.4985 | Source: chartreuse_de_parme_stendhal.md
    Publication: 1839
Catégorie(s): Fiction, Roman...

[3] Score: 0.4709 | Source: chartreuse_de_parme_stendhal.md
    Cette lettre contenait encore cinq ou six pages de détails, elle était écrite en caractères microscopiques sur du papier très fin.
"Tout cela est fort beau et fort bien inventé, se dit Fabrice; je doi...


## 8. Générer une réponse avec le LLM

Utiliser les documents récupérés pour générer une réponse via l'API.

In [32]:
query = "Quel est le titre du roman ?"

print(f"Question: {query}\n")

# Recherche
results = rag.search(query, top_k=5)
context_docs = [doc for doc, score in results]

# Génération
print("Génération de la réponse...\n")
response = rag.generate_with_llm(query, context_docs)

print("="*70)
print("RÉPONSE:")
print("="*70)
print(response)

Question: Quel est le titre du roman ?

Génération de la réponse...

RÉPONSE:
*La Chartreuse de Parme*.


## 9. Fonction helper pour Q&A interactive

Créer une fonction simple pour poser des questions.

In [33]:
import os

def ask(question: str, top_k: int = 10, show_sources: bool = True):
    """Pose une question au système RAG et affiche la réponse."""
    print(f"\n{'='*70}")
    print(f"QUESTION: {question}")
    print(f"{'='*70}\n")

    # Recherche
    results = rag.search(question, top_k=top_k)

    if show_sources:
        print("📚 Sources trouvées:")
        for i, (doc, score) in enumerate(results[:3], 1):
            print(f"  [{i}] {doc['source']} (score: {score:.3f})")
        print("etc.")

    # Génération
    context_docs = [doc for doc, score in results]

    # Écrire la question et le contexte dans un fichier
    output_context_path = os.path.join(DRIVE_DATA_DIR, 'contexte.txt')
    with open(output_context_path, 'w', encoding='utf-8') as f:
        f.write(f"Question: {question}\n\n")
        f.write("Contexte utilisé:\n")
        for doc in context_docs:
            f.write(f"---\nSource: {doc['source']}\n{doc['text']}\n")
        print(f"Question et contexte sauvegardés dans : {output_context_path}")

    response = rag.generate_with_llm(question, context_docs)

    print("💬 RÉPONSE:")
    print("-"*70)
    print(response)
    print("-"*70)

    return response

print("Fonction ask() définie. Utilisez: ask('votre question')")

Fonction ask() définie. Utilisez: ask('votre question')


## 10. Exemples d'utilisation

Testez le système avec différentes questions.

In [19]:
# Exemple 1
ask("Quel est le titre du roman ?")


QUESTION: Quel est le titre du roman ?

📚 Sources trouvées:
  [1] chartreuse_de_parme_stendhal.md (score: 0.663)
  [2] chartreuse_de_parme_stendhal.md (score: 0.491)
  [3] chartreuse_de_parme_stendhal.md (score: 0.436)
etc.
Question et contexte sauvegardés dans : /content/drive/MyDrive/m1_csmi_sgdb/data/contexte.txt
Basé sur le contexte suivant (et uniquement sur ce contexte), réponds à la question de manière concise et précise.

Contexte:
Titre du roman: La Chartreuse de Parme

Publication: 1839
Catégorie(s): Fiction, Roman

- Quelle bassesse à moi! s'était-elle écriée: dire du mal à Fabrice de la femme qu'il aime!...

- Rendez-moi ce papier, dit la duchesse, et, devant lui, elle le brûla à la bougie.
"Il ne faut pas, ajouta-t-elle, que mon nom paraisse si vous êtes pris et exécuté, car il y va de votre tête.

Cette lettre contenait encore cinq ou six pages de détails, elle était écrite en caractères microscopiques sur du papier très fin.
"Tout cela est fort beau et fort bien inventé

'*La Chartreuse de Parme*.'

In [36]:
# Exemple 2
ask("Qui est Clélia ?")


QUESTION: Qui est Clélia ?

📚 Sources trouvées:
  [1] chartreuse_de_parme_stendhal.md (score: 0.611)
  [2] chartreuse_de_parme_stendhal.md (score: 0.605)
  [3] chartreuse_de_parme_stendhal.md (score: 0.594)
etc.
Question et contexte sauvegardés dans : /content/drive/MyDrive/m1_csmi_sgdb/data/contexte.txt
💬 RÉPONSE:
----------------------------------------------------------------------
Clélia est une jeune femme noble et pieuse, amoureuse de Fabrice, mais contrainte par un voeu à la Madone de ne jamais le revoir. Elle sacrifie sa vie pour lui, malgré son amour intense, et maintient une correspondance secrète avec lui en prison.
----------------------------------------------------------------------


'Clélia est une jeune femme noble et pieuse, amoureuse de Fabrice, mais contrainte par un voeu à la Madone de ne jamais le revoir. Elle sacrifie sa vie pour lui, malgré son amour intense, et maintient une correspondance secrète avec lui en prison.'

In [37]:
import os

# Vérifier le contenu du répertoire de données de Google Drive
print(f"Contenu du répertoire {DRIVE_DATA_DIR}:")
if os.path.exists(DRIVE_DATA_DIR):
    print(os.listdir(DRIVE_DATA_DIR))
else:
    print(f"Le répertoire {DRIVE_DATA_DIR} n'existe pas ou n'est pas accessible.")

# Si le problème persiste, vous pouvez tenter de re-monter Google Drive:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)
# print(f"Google Drive re-monté. Répertoire de données: {DRIVE_DATA_DIR}")

Contenu du répertoire /content/drive/MyDrive/m1_csmi_sgdb/data:
['chartreuse_de_parme_stendhal.md', 'contexte.txt']


In [ ]:
# Exemple 3 - Posez votre propre question
ask("Quel est le titre du roman ?")


QUESTION: Quel est le titre du roman ?

📚 Sources trouvées:
  [1] chartreuse_de_parme_stendhal.md (score: 0.603)
  [2] chartreuse_de_parme_stendhal.md (score: 0.582)
  [3] chartreuse_de_parme_stendhal.md (score: 0.581)
etc.
Question et contexte sauvegardés dans : /content/drive/MyDrive/m1_csmi_sgdb/data/contexte.txt
Basé sur le contexte suivant (et uniquement sur ce contexte), réponds à la question de manière concise et précise.

Contexte:
- Quel homme est-ce que Dugnani, vicaire de Saint-Paul?
- Un petit esprit et une grande ambition répondit l'archevêque, peu de scrupules et une extrême pauvreté, car nous en avons des vices!
- Tudieu, monseigneur! s'écria le ministre, vous peignez comme Tacite.

Et il part au galop; son camarade le suit. Tout cela fut fait en un clin d'oeil.

Il fallait que dans son exil à Romagnano Fabrice:
1 Ne manquât pas d'aller à la messe tous les jours, prît pour confesseur un homme d'esprit, dévoué à la cause de la monarchie, et ne lui avouât, au tribunal de 

'Le roman s\'intitule "La Chartreuse de Parme".'

## 11. Changer de provider à la volée

Vous pouvez changer de provider sans recréer l'instance RAG.

In [ ]:
# Exemple: passer à un autre provider
rag.configure_provider("MISTRAL_LARGE")
# rag.configure_provider("IRMA_LLMCODE")
# rag.configure_provider("PALGANIA_QWEN3")

print("Provider actuel:", rag._provider_cfg.name)
print("Pour changer: rag.configure_provider('PROVIDER_NAME')")

Provider reconfiguré: MISTRAL_LARGE | URL: https://api.mistral.ai/v1/chat/completions | Modèle: open-mistral-nemo
Provider actuel: MISTRAL_LARGE
Pour changer: rag.configure_provider('PROVIDER_NAME')


## 12. Sauvegarder des résultats sur Google Drive

Exemple d'écriture de résultats dans Google Drive.

In [ ]:
# Créer un fichier de résultats dans Google Drive
output_path = '/content/drive/MyDrive/rag_results.txt'

with open(output_path, 'w', encoding='utf-8') as f:
    f.write("RÉSULTATS RAG\n")
    f.write("="*70 + "\n\n")

    # Exemple de sauvegarde d'une Q&A
    question = "Qu'est-ce que le RAG ?"
    results = rag.search(question, top_k=3)
    context_docs = [doc for doc, score in results]
    response = rag.generate_with_llm(question, context_docs)

    f.write(f"Question: {question}\n\n")
    f.write(f"Réponse:\n{response}\n\n")
    f.write("Sources:\n")
    for i, (doc, score) in enumerate(results, 1):
        f.write(f"  [{i}] {doc['source']} (score: {score:.3f})\n")

print(f"Résultats sauvegardés dans: {output_path}")